<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [5]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props
    
import numpy as np
from scipy.optimize import root, minimize

In [6]:
p = Props(['Ethanol','Isopropanol', 'Water'])

In [7]:
# Static parameters (Total feed, feed mole fractions, feed temperature and )
c=DotDict()
c.Ftot=10 # Total Feed moles
c.Fz = np.array([1/3, 1/3, 1/3]) # Equimolar feed composition
c.FT = 450 # Feed temperature
c.flashP= 101325 # Flash drum pressure

c.Vy = Comp(c.Fz) # Guess vapor/liquid composition equal to feed
c.Lx = Comp(c.Fz) # Comp - constrains mole fractions to behave like mole fractions!
c.flashT = Range(360, 273.15, c.FT)  # Guess and bounds for flash temperature
c.Vtot = Range(c.Ftot/2, 0., c.Ftot)  # Guess half of feed in vapor
c.Ltot = Range(c.Ftot/2, 0., c.Ftot)

In [8]:
wrap, x, d2, xtod, xtodunk = dtox(c)

In [10]:
def eqs(d):
    r=DotDict()
    V = d.Vy * d.Vtot # Moles of each component = mole fractions * total moles
    L = d.Lx * d.Ltot
    F = d.Fz * d.Ftot
    r.mass_balance = F - V - L # Mass balance for each component (vectors!)

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    FH = p.Hl(nL=F, T=d.FT)
    VH = p.Hv(nV=V, T=d.flashT)
    LH = p.Hl(nL=L, T=d.flashT)
    r.energy_balance = (FH - VH - LH)

    # Raoults with NRTL activity coefficient correction.  One-liner!
    fugL = d.Lx  * p.NRTL_gamma(d.Lx,d.flashT)* p.Pvap(d.flashT)
    fugV = d.Vy * d.flashP
    r.VLE = fugL - fugV
    return np.concatenate([r.mass_balance, np.atleast_1d(r.energy_balance), r.VLE]), r 


In [11]:
eqs_wrapped = wrap(eqs)

In [12]:
sol=minimize(lambda x: 0., x, method='SLSQP', bounds=[(-15,15)]*len(x), constraints=dict(type='eq',fun=eqs_wrapped))

In [14]:
xtod(sol.x,d2)

{'FT': 450,
 'Ftot': 10,
 'Fz': array([0.33333333, 0.33333333, 0.33333333]),
 'Ltot': array(6.02836122),
 'Lx': array([0.32122647, 0.32919072, 0.34958281]),
 'Vtot': array(3.97163878),
 'Vy': array([0.35170977, 0.33962121, 0.30866903]),
 'flashP': 101325,
 'flashT': array(352.85497499)}